# Using `foldseek` to compare AlphaFold structures

## Context

In this notebook we will walkthrough how to use `foldseek` to compare AlphaFold protein structural predictions. The current release of the AlphaFold DB contains >200 million protein predictions from ~1 million phylogenetically diverse species. Here, we will programmitcally access full 'foldomes' from the AlphaFold DB using NCBI taxonomic IDs. We will then go through the `foldseek` pipeline, first creating databases for the foldomes we want to compare, and then running `foldseek` and clustering the results.

---

## Programmatically downloading AlphaFold foldomes

AlphaFold data can be accessed using Google’s command line tool `gsutil`. AlphaFold proteins are named using the convention `proteomes/proteome-tax_id-[TAX ID]-[SHARD ID].tar`. We will replace the `[TAX ID]` portion with the desired NCBI taxonomy IDs, in this case the IDs for <em>Xenopus</em> `[8355]`, mouse `[10090]`, and zebrafish `[7955]`. 

Note: since AlphaFold foldomes can be split across multiple directories, we will use `*` at the end of the file path to download all relevant data. 

Also, note: since this is notebook running R, the command line calls are wrapped in the `system` function. To run in bash, simply use the code inside the parantheses and quotes.

---

Set working directory using R (can also do this in bash using `cd`).

In [24]:
setwd('~/Desktop/foldseek-walkthrough/')

Create directories to save AlphaFold data and `foldseek` files.

In [12]:
system('mkdir foldseek/')
system('mkdir foldseek/dbs/')
system('mkdir foldseek/out/')
system('mkdir data/')

Use `gsutil` to download the three foldomes.

In [1]:
#Xenopus
system('gsutil -m cp gs://public-datasets-deepmind-alphafold/proteomes/proteome-tax_id-8355-* data/' )

#Mouse
system('gsutil -m cp gs://public-datasets-deepmind-alphafold/proteomes/proteome-tax_id-10090-* data/' )

#Zebrafish
system('gsutil -m cp gs://public-datasets-deepmind-alphafold/proteomes/proteome-tax_id-7955-* data/' )

Untar the foldome directories. `foldseek` expects just one directory containing all of the relevant .pdb or .cif files (one per protein structure).

In [25]:
setwd('data/')
system('ls *.tar |xargs -n1 tar -xzf')
setwd('../')

## Running `foldseek`

Create database with `createdb`.

In [26]:
system('foldseek createdb data/ foldseek/dbs/all_foldomesDB')

Compare the protein structures using `search`.

In [ ]:
system('foldseek search foldseek/dbs/all_foldomesDB foldseek/dbs/all_foldomesDB foldseek/out/all_by_all foldseek/out/tmp -a')

Calculate tm scores (`aln2tmscore`) and compile into a .tsv (`createtsv`; useful for downstream analyses).

In [ ]:
#Get tm scores
system('foldseek aln2tmscore foldseek/dbs/all_foldomesDB foldseek/dbs/all_foldomesDB foldseek/out/all_by_all foldseek/out/all_by_all_tmscore')

#Compile into .tsv
system('foldseek createtsv foldseek/dbs/all_foldomesDB foldseek/dbs/all_foldomesDB foldseek/out/all_by_all_tmscore foldseek/out/all_by_all_tmscore.tsv')

Cluster `foldseek` results. `foldseek` has three built in clustering methods:<br> 
greedy (`--cluster-mode 0`)<br> 
blast (`--cluster-mode 1`)<br> 
cdhit (`--cluster-mode 2`)<br> 
Here we will use greedy (tends to generate better/more protein clusters than the others, at least among the species tested so far).

In [ ]:
#Cluster
system('foldseek clust foldseek/dbs/all_foldomesDB foldseek/out/all_by_all foldseek/out/clu --cluster-mode 0 --similarity-type 2')

#Compile into .tsv
system('foldseek createtsv foldseek/dbs/all_foldomesDB foldseek/out/all_by_all foldseek/out/clu foldseek/out/clu_greedy.tsv')